# Using lazy processing

- Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

- For this exercise, we'll be defining a Data Frame (`aa_dfw_df`) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

## Instructions

- Load the Data Frame.
- Add the transformation for `F.lower()` to the `Destination Airport` column.
- Drop the `Destination Airport` column from the Data Frame `aa_dfw_df`. Note the time for these operations to complete.
- Show the Data Frame, noting the time difference for this action to complete.

In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
!pwd

/home/talentum/test-jupyter/P3/M1/SM2/2_Immutabilityandlazyprocessing


In [5]:
from pyspark.sql import functions as F

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('file:///home/talentum/test-jupyter/P3/M1/SM2/2_Immutabilityandlazyprocessing/Dataset/AA_DFW_2018_Departures_Short.csv.gz')
print(aa_dfw_df.printSchema())

# Add the airport column using the F.lower() method
aa_dfw_df1 = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))
print(aa_dfw_df.printSchema())   # Dataframe schema is still the same  - immutable
print(aa_dfw_df1.printSchema())  # New Dataframe schema 


# Drop the Destination Airport column
aa_dfw_df1 = aa_dfw_df1.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df1.show()


root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)

None
root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)

None
root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)
 |-- airport: string (nullable = true)

None
+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2018|         0005|                          498|    hnl|
|       01/01/2018|   

In [8]:
file_path = 'file:///home/talentum/test-jupyter/P3/M1/SM2/2_Immutabilityandlazyprocessing/Dataset/AA_DFW_2018_Departures_Short.csv.gz'

# Schema for actual dataframe
aa_dfw_df = spark.read.format('csv').options(Header=True).load(file_path)
print(aa_dfw_df.printSchema())

# Schema for modified dataframe
print(aa_dfw_df.select('Date (MM/DD/YYYY)','Destination Airport').printSchema())

# Rename the Date (MM/DD/YYYY) -> Date to use it as  df.Date
print(aa_dfw_df.withColumn('Date',aa_dfw_df['Date (MM/DD/YYYY)']).drop(aa_dfw_df['Date (MM/DD/YYYY)']).printSchema())


root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)

None
root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Destination Airport: string (nullable = true)

None
root
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)
 |-- Date: string (nullable = true)

None
